import sys
!{sys.executable} -m pip install music21
!{sys.executable} -m pip install tqdm


In [1]:
import music21
import numpy as np
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import pandas as pd
from keras.utils import np_utils
from collections import defaultdict

from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten

C:\Users\MHarajli\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
with open('encodings/information.pickle','rb') as file:
    DF=pickle.load(file)

In [21]:
DF=DF[DF['Artist']=='chopin']

In [22]:
DF=DF.iloc[[0,1,2,3],:]

In [3]:

with open('SavedModels/vocab.pickle','rb') as file:
    vocab=pickle.load(file)
    
with open('SavedModels/WordToNumber.pickle','rb') as file:
    WordToNumber=pickle.load(file)

with open('SavedModels/NumberToWord.pickle','rb') as file:
    NumberToWord=pickle.load(file)
    
with open('SavedModels/GfToNumber.pickle','rb') as file:
    GfToNumber=pickle.load(file)
with open('SavedModels/NumberToGf.pickle','rb') as file:
    NumberToGf=pickle.load(file)
    
with open('SavedModels/NoteToNumber.pickle','rb') as file:
    NoteToNumber=pickle.load(file)
    
with open('SavedModels/NumberToNote.pickle','rb') as file:
    NumberToNote=pickle.load(file)




In [4]:
def prepare_sequences_gf_or_note(DF, n_vocab,WordToNumber,sequence_length=100): 
    """Given a list of locations for all the midi files in the dataset, this function encodes each song"""
    # Extract the unique pitches in the list of notes.

    network_input = []
    network_output = []
    for i in tqdm(range(len(DF))):
        song=DF.iloc[i,2]
        # create input sequences and the corresponding outputs
        
        for i in range(0, len(song) - sequence_length, 1):
            #we only use the sequence if the last event is not a gf event:
            sequence_in = song[i: i + sequence_length]
            if 'gf' not in str(sequence_in[-1]):
                sequence_out = float('gf' in str(song[i + sequence_length]))
                network_input.append([WordToNumber[char] for char in sequence_in])
                network_output.append(sequence_out)
                

    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [5]:
def prepare_sequences_get_gf(DF, n_vocab,WordToNumber,GfToNumber,sequence_length=100): 
    """Given a list of locations for all the midi files in the dataset, this function encodes each song"""
    # Extract the unique pitches in the list of notes.

    network_input = []
    network_output = []
    for i in tqdm(range(len(DF))):
        song=DF.iloc[i,2]
        # create input sequences and the corresponding outputs
        
        for i in range(0, len(song) - sequence_length, 1):
            #we only use the sequence if the last event is not a gf event:
            sequence_in = song[i: i + sequence_length]
            sequence_out=song[i + sequence_length]
            if 'gf' in str(sequence_out):
                network_input.append([WordToNumber[char] for char in sequence_in])
                network_output.append(GfToNumber[sequence_out])
                

    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [6]:
def prepare_sequences_get_note(DF, n_vocab,WordToNumber,NoteToNumber,sequence_length=100): 
    """Given a list of locations for all the midi files in the dataset, this function encodes each song"""
    # Extract the unique pitches in the list of notes.

    network_input = []
    network_output = []
    for i in tqdm(range(len(DF))):
        song=DF.iloc[i,2]
        # create input sequences and the corresponding outputs
        
        for i in range(0, len(song) - sequence_length, 1):
            #we only use the sequence if the last event is not a gf event:
            sequence_in = song[i: i + sequence_length]
            sequence_out=song[i + sequence_length]
            if 'gf' not in str(sequence_out):
                network_input.append([WordToNumber[char] for char in sequence_in])
                network_output.append(NoteToNumber[sequence_out])
                

    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [7]:
def create_network_gf_or_note(network_in, n_vocab_out): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(20, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(20,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab_out))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    return model

In [8]:
def create_network_get_gf(network_in, n_vocab_out): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(20, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(20,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab_out))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    return model

In [9]:
def create_network_get_note(network_in, n_vocab_out): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(20, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(20,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab_out))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

    return model

In [11]:
#we first need to load the trained models:

#gf_or_note_input,gf_or_note_output=prepare_sequences_gf_or_note(DF,len(WordToNumber),WordToNumber)
#get_gf_input,get_gf_output=prepare_sequences_get_gf(DF,len(WordToNumber),WordToNumber,GfToNumber)
#get_note_input,get_note_output=prepare_sequences_get_note(DF,len(WordToNumber),WordToNumber,NoteToNumber)

from keras.models import load_model
get_gf = load_model('SavedModels/weights.get_gf.hdf5')

gf_or_note = load_model('SavedModels/weights.gf_or_note.hdf5')

get_note = load_model('SavedModels/weights.get_note.hdf5')



In [34]:
#now we have all three models. We start with a sequence:
#We pick the beggining of the first song in our DF: chpn-p1:
encoding=DF.iloc[3,2]
sequenceIn=[]

for word in encoding[:100]:
    sequenceIn.append(WordToNumber[word])
FullSequence=sequenceIn

for i in range(300):
    if 'gf' in str(NumberToWord[sequenceIn[-1]]):
        
        next_note=get_note.predict(np.reshape(sequenceIn,(1,100,1))/len(vocab))
        next_note=np.argmax(next_note)
        next_note=NumberToNote[next_note]
        next_note=WordToNumber[next_note]
        FullSequence.append(next_note)
    else:
        
        nextStep=gf_or_note.predict(np.reshape(sequenceIn,(1,100,1))/len(vocab))
        nextStep=np.argmax(nextStep)
        
        #we get nextStep=1 if next step is a gf and 0 if note
        if nextStep==0:
            next_note=get_note.predict(np.reshape(sequenceIn,(1,100,1))/len(vocab))
            next_note=np.argmax(next_note)
            next_note=NumberToNote[next_note]
            next_note=WordToNumber[next_note]
            FullSequence.append(next_note)
        else:
            next_gf=get_gf.predict(np.reshape(sequenceIn,(1,100,1))/len(vocab))
            next_gf=np.argmax(next_gf)
            next_gf=NumberToGf[next_gf]
            
            next_gf=WordToNumber[next_gf]
            FullSequence.append(next_gf)
    sequenceIn=FullSequence[-100:]
            
Composition=[]            
for number in FullSequence:
    Composition.append(NumberToWord[number])
        

In [35]:
#next we write a function that takes in a sequence of events and changes it into a midi file:
def write_midi(composition,filename):
    MidiSequence=[]
    Offset=0
    for word in composition:
        if 'gf' in str(word):
            Offset+=float(word[2:])
        else:
            new_note=music21.note.Note(int(word))
            new_note.offset=Offset
            new_note.storedInstrument=music21.instrument.Piano()
            MidiSequence.append(new_note)
            
    midi_stream=music21.stream.Stream(MidiSequence)
    midi_stream.write('midi', fp=filename)    
    
    return 1


In [36]:
write_midi(Composition,'Samples/ChopinOnly.mid')

1

In [24]:
for i in range(len(Composition)):
    if encoding[i]==Composition[i]:
        print('Song Goes: '+str(encoding[i])+' and we guess: '+str(Composition[i])+' at index: '+str(i))
    else:
        print('Song Goes: '+str(encoding[i])+' and we guess: '+str(Composition[i])+' at index: '+str(i)+'<---------')
        

Song Goes: 36 and we guess: 36 at index: 0
Song Goes: gf0.25 and we guess: gf0.25 at index: 1
Song Goes: 43 and we guess: 43 at index: 2
Song Goes: 55 and we guess: 55 at index: 3
Song Goes: gf0.0833 and we guess: gf0.0833 at index: 4
Song Goes: 52 and we guess: 52 at index: 5
Song Goes: 60 and we guess: 60 at index: 6
Song Goes: gf0.1667 and we guess: gf0.1667 at index: 7
Song Goes: 64 and we guess: 64 at index: 8
Song Goes: 67 and we guess: 67 at index: 9
Song Goes: gf0.1667 and we guess: gf0.1667 at index: 10
Song Goes: 60 and we guess: 60 at index: 11
Song Goes: gf0.0833 and we guess: gf0.0833 at index: 12
Song Goes: 57 and we guess: 57 at index: 13
Song Goes: 69 and we guess: 69 at index: 14
Song Goes: gf0.25 and we guess: gf0.25 at index: 15
Song Goes: 35 and we guess: 35 at index: 16
Song Goes: gf0.25 and we guess: gf0.25 at index: 17
Song Goes: 43 and we guess: 43 at index: 18
Song Goes: 55 and we guess: 55 at index: 19
Song Goes: gf0.0833 and we guess: gf0.0833 at index: 20
So

Song Goes: gf0.25 and we guess: gf0.25 at index: 321
Song Goes: 60 and we guess: 43 at index: 322<---------
Song Goes: 74 and we guess: 55 at index: 323<---------
Song Goes: gf0.0833 and we guess: gf0.0833 at index: 324
Song Goes: 67 and we guess: 55 at index: 325<---------
Song Goes: 76 and we guess: 60 at index: 326<---------
Song Goes: gf0.1667 and we guess: gf0.1667 at index: 327
Song Goes: 79 and we guess: 64 at index: 328<---------
Song Goes: 86 and we guess: 67 at index: 329<---------
Song Goes: gf0.1667 and we guess: gf0.1667 at index: 330
Song Goes: 76 and we guess: 64 at index: 331<---------
Song Goes: gf0.0833 and we guess: gf0.0833 at index: 332
Song Goes: 72 and we guess: 57 at index: 333<---------
Song Goes: 84 and we guess: 69 at index: 334<---------
Song Goes: gf0.25 and we guess: gf0.25 at index: 335
Song Goes: 42 and we guess: 36 at index: 336<---------
Song Goes: gf0.25 and we guess: gf0.25 at index: 337
Song Goes: 51 and we guess: 43 at index: 338<---------
Song Goe

In [ ]:
encoding=DF.iloc[0,2]
sequenceIn=[]

for word in encoding[:100]:
    sequenceIn.append(WordToNumber[word])
get_gf.predict(np.reshape(sequenceIn,(1,100,1))/len(vocab))

In [ ]:
with open('SavedModels/just.testing.pickle','rb') as file:
    trialIn=pickle.load(file)

In [ ]:
get_gf.predict(np.reshape(trialIn,(1,100,1))/len(vocab))